# M3 Model Discovery and Extraction

This notebook demonstrates how to access HEC-HMS projects from the Harris County Flood Control District (HCFCD) M3 Models.

**M3 Models** are FEMA-effective H&H (Hydrology & Hydraulics) models for major bayous and watersheds in the Houston, Texas region. Each model contains:
- **HEC-HMS projects**: Hydrologic models for runoff generation
- **HEC-RAS projects**: Hydraulic models for river routing

## What You'll Learn

1. Listing available M3 models and HMS projects
2. Querying project metadata
3. Finding projects by channel name
4. Extracting HMS projects for use
5. Integration with hms-commander workflows

In [ ]:
# pip install hms-commander

**For Development**: If working on hms-commander source code, use the `hmscmdr_local` conda environment (editable install) instead of pip install.

**Note**: This notebook **requires M3 data download**. The first extraction will download model archives from the HCFCD model library (~25-100 MB per model).

## Setup

In [ ]:
from pathlib import Path

from hms_commander import HmsM3Model, init_hms_project, __version__
import pandas as pd

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print(f"hms-commander v{__version__}")

## 1. List Available M3 Models

M3 Models are identified by single letters (A-W, excluding V). Each model covers a specific watershed or bayou in Harris County.

In [ ]:
# List all M3 models that contain HMS projects
models = HmsM3Model.list_models()
print(f"Total M3 Models with HMS projects: {len(models)}\n")
models[['model_id', 'name', 'hms_project_count', 'effective_date']]

## 2. List HMS Projects

Each M3 model contains one or more HMS projects. The catalog includes metadata about each project.

In [ ]:
# List all HMS projects across all models
all_projects = HmsM3Model.list_projects()
print(f"Total HMS projects: {len(all_projects)}\n")
all_projects[['model_id', 'model_name', 'unit_id', 'hms_version', 'loss_method', 'transform_method']].head(15)

In [ ]:
# List projects for a specific model (Model G has the most)
san_jacinto_projects = HmsM3Model.list_projects(model_id='G')
print(f"San Jacinto River (Model G) HMS projects: {len(san_jacinto_projects)}\n")
san_jacinto_projects[['unit_id', 'hms_file', 'hms_version']]

## 3. Get Model and Project Information

Get detailed metadata about specific models and projects.

In [ ]:
# Get detailed info about Brays Bayou (Model D)
model_info = HmsM3Model.get_model_info('D')

print("=== Model D: Brays Bayou ===")
print(f"Name: {model_info['name']}")
print(f"Effective Date: {model_info['effective_date']}")
print(f"Download Size: {model_info['size_gb']} GB")
print(f"Primary Channels: {model_info['primary_channels']}")
print(f"HMS Projects: {model_info['hms_projects']}")

In [ ]:
# Get detailed info about a specific HMS project
project_info = HmsM3Model.get_project_info('D', 'D100-00-00')

print("=== Project D100-00-00 ===")
for key, value in project_info.items():
    print(f"{key}: {value}")

## 4. Find Projects by Channel Name

You can find M3 models and HMS projects by searching for channel names.

In [ ]:
# Find HMS project by channel name
channels_to_find = [
    'BRAYS BAYOU',
    'BUFFALO BAYOU',
    'WHITE OAK BAYOU',
    'GREENS BAYOU',
    'CLEAR CREEK'
]

print("Channel Name Lookups:")
print("-" * 60)
for channel in channels_to_find:
    result = HmsM3Model.get_project_by_channel(channel)
    if result:
        model_id, unit_id = result
        model_name = HmsM3Model.MODELS[model_id]['name']
        print(f"{channel:20} -> Model {model_id} ({model_name}), Unit {unit_id}")
    else:
        print(f"{channel:20} -> Not found")

## 5. Catalog Statistics

Get summary statistics about the HMS project catalog.

In [ ]:
# Get catalog statistics
stats = HmsM3Model.get_statistics()

print("=== M3 HMS Catalog Statistics ===")
print(f"Total HMS Projects: {stats['total_projects']}")
print(f"Total M3 Models: {stats['total_models']}")
print(f"\nModels with Most HMS Projects:")
for model_id, count in stats['models_with_most_projects'].items():
    print(f"  Model {model_id}: {count} projects")
print(f"\nHMS Versions:")
for version, count in stats['hms_versions'].items():
    print(f"  {version}: {count} projects")
print(f"\nLoss Methods:")
for method, count in stats['loss_methods'].items():
    print(f"  {method}: {count} projects")

## 6. Extract HMS Project

To use an HMS project, extract it from the M3 archive. This downloads the model zip and extracts the HMS portion.

**Note**: First extraction downloads ~25-100 MB per model.

In [ ]:
# Extract Brays Bayou HMS project
# This downloads ~32 MB and extracts the HMS files

path = HmsM3Model.extract_project(
    'D', 
    'D100-00-00',
    output_path=Path.cwd() / 'hms_example_projects' / 'm3_brays_bayou'
)
print(f"Extracted to: {path}")

# List extracted files
hms_files = list(Path(path).glob('*.hms'))
print(f"\nHMS project file: {hms_files[0].name if hms_files else 'Not found'}")
print(f"\nAll files ({len(list(Path(path).glob('*')))} total):")
for f in sorted(Path(path).glob('*'))[:10]:
    print(f"  {f.name}")
if len(list(Path(path).glob('*'))) > 10:
    print("  ...")

In [ ]:
# Check if a project is already extracted
is_extracted = HmsM3Model.is_project_extracted('D', 'D100-00-00')
print(f"D100-00-00 extracted: {is_extracted}")

## 7. Integration with hms-commander Workflow

Once extracted, use the HMS project with standard hms-commander operations.

**Important**: M3 HMS projects use HMS 3.x format (version 3.3 or 3.4). When generating Jython scripts, you must use `python2_compatible=True`.

In [ ]:
# Initialize the extracted project
if path and Path(path).exists():
    hms = init_hms_project(path)
    
    print(f"Project: {hms.project_name}")
    print(f"HMS Version: {hms.hms_version}")
    print(f"\nBasin models: {len(hms.basin_df)}")
    print(f"Met models: {len(hms.met_df)}")
    print(f"Control specs: {len(hms.control_df)}")
    print(f"Runs: {len(hms.run_df)}")
    
    print(f"\nAvailable runs:")
    for run_name in hms.list_run_names():
        print(f"  - {run_name}")

### Example Execution Workflow (HMS 3.x)

```python
from hms_commander import HmsM3Model, init_hms_project, HmsJython, HmsResults

# 1. Extract project (downloads ~32 MB)
path = HmsM3Model.extract_project('D', 'D100-00-00')

# 2. Initialize project
hms = init_hms_project(path)
print(f"Project: {hms.project_name}, Version: {hms.hms_version}")
print(f"Available runs: {hms.list_run_names()}")

# 3. Generate execution script (MUST use python2_compatible for HMS 3.x)
script = HmsJython.generate_compute_script(
    project_path=path,
    run_name="1PCT",
    python2_compatible=True  # Required for M3 HMS projects!
)

# 4. Execute (requires HMS 3.x installation)
# HmsCmdr.compute_run("1PCT", python2_compatible=True)

# 5. Extract results
dss_file = path / "D100_00_00.dss"
peaks = HmsResults.get_peak_flows(dss_file)
print(peaks)
```

## 8. HMS-RAS Integration

M3 Models contain both HMS (hydrology) and RAS (hydraulics) projects. Use `ras-commander` for RAS operations.

**Typical Workflow**:
1. Run HMS to generate flow hydrographs
2. Import HMS DSS results as RAS upstream boundary conditions
3. Run RAS for water surface profiles

In [ ]:
# Cross-reference HMS and RAS for same watershed
print("=== Brays Bayou (Model D) - HMS and RAS Projects ===")
print()

# HMS side
hms_info = HmsM3Model.get_model_info('D')
print(f"HMS Projects: {hms_info['hms_projects']}")
print(f"  - 1 HMS project covers entire watershed hydrology")
print()

# RAS side (informational - requires ras-commander)
print("RAS Reaches (via ras-commander M3Model):")
print("  D100-00-00, D109-00-00, D111-00-00, D112-00-00,")
print("  D118-00-00, D120-00-00, D122-00-00, D124-00-00,")
print("  D126-00-00, D129-00-00, D132-00-00, D133-00-00,")
print("  D139-00-00, D140-00-00, D142-00-00, D144-00-00")
print(f"  - 16 RAS reaches for detailed hydraulics")

## Summary

`HmsM3Model` provides access to 42 HMS projects across 22 HCFCD M3 Models:

| Feature | Description |
|---------|-------------|
| Models | 22 M3 models (A-W) with HMS content |
| Projects | 42 HMS projects total |
| HMS Version | Mostly 3.3 (requires `python2_compatible=True`) |
| Design Storms | 0.2%, 1%, 2%, 10% AEP typically |
| Unit System | Mostly Metric |

**Key Methods**:

| Method | Purpose |
|--------|--------|
| `list_models()` | List M3 models with HMS projects |
| `list_projects()` | List all HMS projects |
| `get_model_info()` | Get model metadata |
| `get_project_info()` | Get project metadata |
| `get_project_by_channel()` | Find by channel name |
| `extract_project()` | Download and extract HMS project |

## Next Steps

- **09_m3_conversion.ipynb**: Convert HMS 3.x to 4.x with QAQC
- **07_execution_jython.ipynb**: Advanced execution patterns
- **HCFCD Resources**: https://www.m3models.org/